## Ершов Вячеслав ПИ22-5. Оптимизация выполнения кода, векторизация, Numba

Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [8]:
import random
N, A, B = 1000000, [random.randint(0, 1000) for i in range(N)], [a + 100 for a in A]
sum(B) / N

599.75218

2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [12]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.rand(2000000, 4), columns=['1', '2', '3', '4'])
df['key'] = [chr(np.random.randint(97, 122)) for i in range(2000000)]
df[df['key'].str.contains('^[a-eA-E]')]

,1,2,3,4,key
0,0.103603,0.329251,0.301553,0.676110,d
5,0.250824,0.750851,0.451737,0.027784,b
15,0.526242,0.643965,0.531371,0.250244,b
18,0.644902,0.680996,0.424857,0.138413,c
20,0.192212,0.317171,0.349968,0.592542,e
...,...,...,...,...,...
1999976,0.949529,0.804552,0.540118,0.354177,b
1999984,0.694213,0.917859,0.773019,0.059782,a
1999990,0.254923,0.184070,0.044317,0.727526,d
1999992,0.089897,0.389757,0.618148,0.438823,d


## Лабораторная работа 3

1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [14]:
import pandas as pd
import time
recipes = pd.read_csv('recipes_sample.csv', index_col=0, delimiter=',')
reviews = pd.read_csv('reviews_sample.csv', index_col=0, delimiter=',')
reviews['date'] = pd.to_datetime(reviews['date'])
def mean1(reviews):
    sumall = 0
    counter = 0
    for index, row in reviews.iterrows():
        if row['date'].year == 2010:
            sumall += row['rating']
            counter += 1
    return sumall / counter
def mean2(reviews):
    sumall = 0
    counter = 0
    for index, row in reviews.loc[reviews['date'].dt.year == 2010].iterrows():
        sumall += row['rating']
        counter += 1
    return sumall / counter
def mean3(reviews):
    filtered_reviews = reviews.loc[reviews['date'].astype(str).str.contains('2010')]
    return filtered_reviews['rating'].mean()
print(f'1) {mean1(reviews)}')
print(f'2) {mean2(reviews)}')
print(f'3) {mean3(reviews)}')
print('\n')
start_time = time.time()
res1 = mean1(reviews)
end_time = time.time()
print(f'mean1: {res1}, time - {end_time - start_time}')
start_time = time.time()
res2 = mean2(reviews)
end_time = time.time()
print(f'mean2: {res2}, time - {end_time - start_time}')
start_time = time.time()
res3 = mean3(reviews)
end_time = time.time()
print(f'mean3: {res3}, time - {end_time - start_time}')

1) 4.4544402182900615
2) 4.4544402182900615
3) 4.4544402182900615


mean1: 4.4544402182900615, time - 4.484178304672241
mean2: 4.4544402182900615, time - 0.4145667552947998
mean3: 4.4544402182900615, time - 0.7855618000030518


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

In [15]:
print('Вариант A, так как мы итерируемся абсолютно по всем строкам, а в других вариантах мы проходимся\
только по отфильтрованным (только отзывы за 2010 год) строкам, что значительно уменьшает количество обрабатываемых\
данных, вследствие чего сильно увеличивает время выполнения функции')

Вариант A, так как мы итерируемся абсолютно по всем строкам, а в других вариантах мы проходимсятолько по отфильтрованным (только отзывы за 2010 год) строкам, что значительно уменьшает количество обрабатываемыхданных, вследствие чего сильно увеличивает время выполнения функции


In [19]:
!pip install line_profiler
%load_ext line_profiler
%lprun -f mean1 mean1(reviews)
def mean1_2(reviews):
    filtered_reviews = reviews[reviews['date'].dt.year == 2010]
    count = len(filtered_reviews)
    if count > 0:
        return filtered_reviews['rating'].apply(lambda x: float(x)).sum() / count
    else:
        return None
%lprun -f mean1_2 mean1_2(reviews)
print('Таким образом, используя лямбда-функцию и метод apply, мы сокращаем время выполнения программы, так как\
в памяти устройства во время выполнения функции не сохраняются данные, а в предыдущей версии функции сохранялись')

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
Таким образом, используя лямбда-функцию и метод apply, мы сокращаем время выполнения программы, так какв памяти устройства во время выполнения функции не сохраняются данные, а в предыдущей версии функции сохранялись


In [ ]:
Timer unit: 1e-07 s
Total time: 0.020832 s
Could not find file C:\Users\224995\AppData\Local\Temp/ipykernel_1728/1939468982.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.
Line 
==============================================================
     6                                           
     7         1     116645.0 116645.0     56.0  
     8         1         37.0     37.0      0.0  
     9         1          3.0      3.0      0.0  
    10         1      91635.0  91635.0     44.0

3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

In [18]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        words = review.split(' ')
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count
%lprun -f get_word_reviews_count get_word_reviews_count(reviews) 
from collections import defaultdict
def get_word_reviews_count2(df):
    word_reviews = defaultdict(list) 
    word_reviews_count = defaultdict(int) 
    for _, row in df.dropna(subset=['review']).iterrows():
        recipe_id, review = row['recipe_id'], row['review']
        for word in review.split():
            word_reviews[word].append(recipe_id)
    for word in word_reviews:
        word_reviews_count[word] = len(word_reviews[word])
%lprun -f get_word_reviews_count2 get_word_reviews_count2(reviews)

In [ ]:
Timer unit: 1e-07 s
Total time: 22.8831 s
Could not find file C:\Users\224995\AppData\Local\Temp/ipykernel_1728/2746826856.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.
Line 
==============================================================
    23                                           
    24         1         16.0     16.0      0.0  
    25         1          6.0      6.0      0.0  
    26    126679  138605885.0   1094.2     60.6  
    27    126679   34074469.0    269.0     14.9  
    28   6589870   21515723.0      3.3      9.4  
    29   6589870   33225089.0      5.0     14.5  
    30    164271     472519.0      2.9      0.2  
    31    164271     937480.0      5.7      0.4